In [1]:
from pymongo import MongoClient
import folium
import pandas as pd

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
def getLocation(comp,i):
    latitude = comp['offices'][i]['latitude']
    longitude = comp['offices'][i]['longitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [5]:
def getName(comp,i):
    name = comp['name']
    return name

I am going to filter all companies that have been founded after 2003, that have raised at least 1M dollars and that have a category code similar to a gaming company.

In [6]:
pipeline = [
    { "$unwind": "$offices"},
    {"$match":{ "$and": [  {"founded_year":{"$gt":2003}},
            {"funding_rounds.raised_amount":{"$gt":1000000}}, { "$or":[{"category_code":"web"},
            {"category_code":"software"},{"category_code":"games_video"},
            {"category_code":"hardware"},{"category_code":"mobile"},{"category_code":"music"},
            {"category_code":"photo_video"},{"category_code":"design"}]}] }}
#    {"$match":"}
    ]
results = list(coll.aggregate(pipeline))

In [7]:
name = []
city = []
country_code = []
longitude = []
latitude = []

for e in results:
    name.append(e["name"])
    city.append(e['offices']["city"])
    country_code.append(e['offices']["country_code"])
    for c in e["offices"].items():
        if c[0]=="longitude":
            longitude.append(c[1])
        elif c[0]=="latitude":
            latitude.append(c[1])

In [8]:
print(len(name),len(city),len(country_code),len(longitude),len(latitude))

1211 1211 1211 1211 1211


In [9]:
myData = {"name":name, "city":city, "country":country_code, "latitude":latitude, "longitude":longitude}

In [10]:
startup_df = pd.DataFrame(data=myData)

In [11]:
startup_df

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
...,...,...,...,...,...
1206,Antix Labs,Berkshire,GBR,NaN,NaN
1207,Incuity Software,Mission Viejo,USA,33.664087,-117.632519
1208,Bump Technologies,Mountain View,USA,NaN,NaN
1209,Tiny Speck,Vancouver,CAN,49.282455,-123.109217


Here I could use geocoding to obtain the null coordinates with the address of each offices, but for simplifying purposes, I am going to get rid of them.

In [12]:
startup_clean = startup_df.dropna()

In [13]:
startup_clean

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
...,...,...,...,...,...
1195,Clavis Technology,Cambridge,USA,42.362663,-71.084153
1197,Equiendo,Nangor Road,IRL,53.337074,-6.251891
1205,Continuity Engine,New Haven,USA,41.327918,-73.243552
1207,Incuity Software,Mission Viejo,USA,33.664087,-117.632519


In [14]:
import folium

In [15]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)
for index, row in startup_clean.iterrows():
    folium.Marker((row['latitude'],row['longitude']),
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    


In [16]:
map_city

## APIs

In [17]:
import os
from dotenv import load_dotenv
import requests
load_dotenv()


True

In [26]:
def googleRequestAuthorized(lat,lon,r,keyword):

    # Function
    authToken = os.getenv("GOOGLE_API_TOKEN")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}".format(lat,lon,r,keyword,authToken)

    res = requests.get(url)
    data = res.json()
    return data

In [47]:
starbucks = googleRequestAuthorized("40.723731","-73.996431","200","starbucks")

In [48]:
vegan = googleRequestAuthorized("40.723731","-73.996431","200","vegan+restaurant")

In [49]:
club = googleRequestAuthorized("40.723731","-73.996431","500","club")

In [50]:
airport = googleRequestAuthorized("40.723731","-73.996431","20000","airport")

In [51]:
kindergarten = googleRequestAuthorized("40.723731","-73.996431","600","kindergarten")

In [63]:
l = kindergarten['results']

In [64]:

print(l[0]['name'])
print(l[0]['geometry']['location']['lat'])
print(l[0]['geometry']['location']['lng'])

Neighborhood School
40.7244039
-73.9863598


In [ ]:
for i in range(len(startup_clean)) : 
    isbn = startup_clean.loc[i, "isbn13"]
    book = githubRequestAuthorized(isbn)
    try:
        startup_clean.loc[i, 'Genre'] = book['items'][0]['volumeInfo']['categories'][0]
        
    except:
        startup_clean.loc[i, 'Genre'] = 'Others'